## Importing Libraries:

In [1]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from cnnutils import *

%matplotlib inline
np.random.seed(1)

import warnings 
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

NameError: name 'load_dataset' is not defined

In [ ]:
# Example of a picture
index = 6
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

In [ ]:
X_train = X_train_orig / 255 
X_test = X_test_orig / 255
y_train = convert_to_one_hot(y_train_orig, 6).T
y_test = convert_to_one_hot(y_test_orig, 6).T

print('Number of Training examples: ' + str(X_train[0].shape))
print('Number o Testing examples: ' + str(X_test[0].shape))
print('X_train shape :' + str(X_train.shape))
print('X_test shape: ' + str(X_test.shape))
print('y_train shape: ' + str(y_train.shape))
print('y_test shape: ' + str(y_test.shape))

conv_layers = {}

In [15]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for Tensorflow sessions 
    
    n_H0 -- height of an image 
    n_W0 -- width of an image 
    n_C0 -- number of channels 
    n_y -- numbers of labels / classes 
    
    Returns:
    
    X -- placeholder for data input, of the shape [None, n_H0, n_W0, n_C0] and dtype float 
    y -- plachehoder for the input label, of the shape [None, n_C] and dtype float 
    
    """
    X = tf.placeholders(tf.float32, [None, n_H0, n_W0, n_C0])
    y = tf.placeholders(tf.float32, [None, n_y])
    
    return X, y

In [ ]:
X, y = create_placeholders(64, 64, 3, 6)

print('X = ', str(X))
print('y = ', str(y))

In [ ]:
def initializa_parameters():
    
    tf.set_random_seed(1)
    
    W1 = tf.get_variable("W1", [4, 4, 3, 8], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", [2, 2, 8, 16], initializer=tf.contrib.layers.xavier_initializer(seed=0))
    
    parameters = {'W1': W1,
                 'W2': W2}
    
    return parameters

In [ ]:
tf.reset_default_graph() # clears the default graph stack and resets the global default graph

with tf.Session as sess_test: # starts the session and runs Tensorflow operations
    parameters = initialize_parameters()
    init = tf.global_variable_initializer() # initializes global variables 
    sess_test.run(init)
    
    print('W1', str(parameters['W1'].eval()[1,1,1]))
    print('W2', str(parameters['W2'].eval()[1,1,1]))

In [3]:
import numpy as np 

y = np.random.randn(4,6)

print(y)

[[ 0.11965718  0.76097522 -0.15622847 -0.14938278 -0.81229329  1.41810875]
 [-0.17224219  1.83776388 -2.06872358  0.90547537  1.47696846  0.39939721]
 [-1.12241691  0.77380528 -0.74917192 -1.87323102 -1.05564885 -1.00743207]
 [ 0.28079674  0.96892259  0.66151353  1.50333779 -0.24210554 -0.49013245]]


In [19]:
def model(X_Train, Y_Train, X_test, Y_test, learning_rate=0.009, 
          num_epochs=100, minibatch_size=64, print_cost=True):
    
    ops.reset_default_graph()
    tf.set_random_seed(1)
    seed = 3
    m, n_H0, n_W0, n_C0 = X_Train.shape
    n_y = Y_Train.shape[1]
    costs = []
    
    # initialize placeholders 
    X, Y = create_placehoders(n_H0, n_W0, n_C0, n_y)
    
    # initializer parameters 
    parameters = initialize_parameters()
    
    # forward propagation 
    Z3 = forward_propagation(X, parameters)
    
    # compute cost 
    cost = compute_cost(Z3, Y)
    
    # initialize the optimizer 
    optimizer = optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # initializing the variables 
    init = tf.global_variable_initializer()
    
    # running he session
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(num_epochs):
            
            minibatch_cost = 0
            num_minibatches = int(m / minibatch_size)
            seed += 1
            
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            
            for minibatch in minibatches:
                minibatch_X, minibatch_Y = minibatches 
                
                _, temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, y:minibatch_Y})
                
                minibatch_cost += temp_cost / num_minibatches
                
            if print_cost == True and epoch % 5 == 0:
                print('Cost after epoch %i: %f' % (epoch, minibatch_cost))
                
            if print_cost = True and epoch % 1 == 0:
                costs.append(minibatch_cost)
                
            # plot the results 
            plt.plot(np.squeeze(costs))
            plt.xlabel('Cost')
            plt.ylabel('Iterations')
            plt.title('Learning Rate = ' + str(learning_rate))
            
            plt.show()
            
            # Calculate the correct predictions 
            predict_op = tf.argmax(Z3, 1)
            correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))

            # Calculate accuracy on the test set
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print(accuracy)
            train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
            test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
            print("Train Accuracy:", train_accuracy)
            print("Test Accuracy:", test_accuracy)

            return train_accuracy, test_accuracy, parameters

SyntaxError: invalid syntax (<ipython-input-19-54944e67a41f>, line 51)